<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2023/Team05/blob/main/spark_jobs_mark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
#install spark. we are using the one that uses hadoop as the underlying scheduler.
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf  spark-3.2.4-bin-hadoop3.2.tgz
!ls -l

#Provides findspark.init() to make pyspark importable as a regular library.
os.environ["SPARK_HOME"] = "spark-3.2.4-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

total 294140
drwxr-xr-x  1 root root      4096 Apr 27 13:35 sample_data
drwxr-xr-x 13 1000 1000      4096 Apr  9 21:17 spark-3.2.4-bin-hadoop3.2
-rw-r--r--  1 root root 301183180 Apr  9 21:35 spark-3.2.4-bin-hadoop3.2.tgz


In [4]:
!pip install -q findspark pyspark
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!cat /content/drive/MyDrive/Big-Data-Final/nfd_incidents_xd_seg.parquet

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

## Average response time + variance by SegID

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("read_parquet_file").getOrCreate()

df = spark.read.parquet("/content/drive/MyDrive/Big-Data-Final/nfd_incidents_xd_seg.parquet")


In [11]:
print(df.show(5))

+--------------------+-----------+------------+-------------+--------------------+--------------------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+
|         ID_Original|   latitude|   longitude|emdCardNumber|            time_utc|          time_local|response_time_sec|day_of_week|weekend_or_not|            geometry|Incident_ID|       Dist_to_Seg|      XDSegID|
+--------------------+-----------+------------+-------------+--------------------+--------------------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+
|ObjectId(59d3a819...|36.03722849|-86.78324314|         29B5|2017-01-01 07:59:...|2017-01-01 01:59:...|            268.0|          6|             1|POINT (-86.783243...|         10|13.550370382347305|1.524393684E9|
|ObjectId(59d3a819...|36.03741402|-86.78657189|        29D2P| 2017-01-01 08:30:55| 2017-01-01 02:30:55|            512.0|          6|       

In [ ]:
#HERE WE HAVE TO CREATE THE CENSUS TRACT PER ROW

In [14]:
from pyspark.sql.functions import avg

# Read the input file as a Spark dataframe
# df = spark.read.format("csv").option("header", "true").load("input.csv")

# Group the dataframe by XDSegID and compute the average response time
result_df = df.groupBy("XDSegID").agg(avg("response_time_sec").alias("AvgResponseTime"))

# Write the result to a CSV file
result_df.write.format("csv").option("header", "true").mode("overwrite").save("output.csv")

In [15]:
result_df.show()

+-------------+------------------+
|      XDSegID|   AvgResponseTime|
+-------------+------------------+
|1.524470198E9|             349.2|
| 1.52446673E9|             404.0|
| 4.41552617E8| 297.6666666666667|
|  4.5042845E8|             582.5|
|1.524609417E9|             407.6|
|1.524475884E9|             337.0|
| 4.29334846E8|            490.25|
|1.524322123E9|             154.5|
| 4.49620825E8|             553.0|
|1.524554221E9|             410.0|
| 4.49621869E8|             798.0|
| 4.49629818E8|             148.5|
| 1.56811863E8|             405.0|
|1.524576615E9|             496.0|
| 4.29334039E8|             449.6|
| 1.56551371E8|             256.0|
| 1.56121138E8|             310.8|
|1.524303919E9|           242.625|
|1.524378121E9|476.14285714285717|
|1.524474299E9|             449.5|
+-------------+------------------+
only showing top 20 rows



In [17]:
from pyspark.sql.functions import col, variance

# group by XDSegID and calculate variance of ResponseTime
df_agg = df.groupBy("XDSegID").agg(variance(col("response_time_sec")).alias("Variance"))

# show the results
df_agg.show()

+-------------+------------------+
|      XDSegID|          Variance|
+-------------+------------------+
|1.524470198E9|           51834.2|
| 1.52446673E9| 68509.06666666667|
| 4.41552617E8| 4837.333333333333|
|  4.5042845E8|          180600.5|
|1.524609417E9|           23415.3|
|1.524475884E9|              null|
| 4.29334846E8|         124896.25|
|1.524322123E9|             364.5|
| 4.49620825E8|           18050.0|
|1.524554221E9|              null|
| 4.49621869E8|              null|
| 4.49629818E8|           42340.5|
| 1.56811863E8|              null|
|1.524576615E9|              null|
| 4.29334039E8|11660.300000000001|
| 1.56551371E8|            2584.0|
| 1.56121138E8|           20254.7|
|1.524303919E9|14160.916666666664|
|1.524378121E9|18207.476190476194|
|1.524474299E9|           19012.5|
+-------------+------------------+
only showing top 20 rows



## Weather correlation on the the incidents and response time distribution

In [19]:
from pyspark.sql.functions import min, max, date_format

min_local_time = df.select(min(date_format('time_local', 'yyyy-MM-dd HH:mm:ss'))).collect()[0][0]
max_local_time = df.select(max(date_format('time_local', 'yyyy-MM-dd HH:mm:ss'))).collect()[0][0]

print("Minimum local time: ", min_local_time)
print("Maximum local time: ", max_local_time)


Minimum local time:  2017-01-01 01:59:29
Maximum local time:  2021-03-02 07:45:57


In [20]:
weather_df = spark.read.parquet("/content/drive/MyDrive/Big-Data-Final/weather_tn.parquet/year=2017/month=1/part-00039-0c8142a9-ffb0-4186-8d5f-060d4d2afe7d.c000.snappy.parquet")

In [22]:
weather_df.show(5)

+------------+-------------+-----------+-------------------+----+--------+-------------------+---+----------+--------+----------+---------+-----+-------+-----+----+---+-------+--------+---+---+----+---+-------------+----+------+------+-----------+----+-----+---------------+-----------------------+------------------------+-----------+----------+
|  station_id|start_date_st|end_date_st|    timestamp_local|  rh|wind_spd|      timestamp_utc|pod|       slp|app_temp|elev_angle|solar_rad| pres|h_angle|dewpt|snow| uv|azimuth|wind_dir|ghi|dhi| vis|dni|     datetime|temp|precip|clouds|         ts|icon| code|    description|gps_coordinate_latitude|gps_coordinate_longitude| spatial_id|      days|
+------------+-------------+-----------+-------------------+----+--------+-------------------+---+----------+--------+----------+---------+-----+-------+-----+----+---+-------+--------+---+---+----+---+-------------+----+------+------+-----------+----+-----+---------------+-----------------------+--------

In [44]:
df.show(5)

+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|         ID_Original|   latitude|   longitude|emdCardNumber|            time_utc|time_local|response_time_sec|day_of_week|weekend_or_not|            geometry|Incident_ID|       Dist_to_Seg|      XDSegID|     time_local_str|    timestamp_local|      date|
+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|ObjectId(59d3a819...|36.03722849|-86.78324314|         29B5|2017-01-01 07:59:...|1483235969|            268.0|          6|             1|POINT (-86.783243...|         10|13.550370382347305|1.524393684E9|2017-01-01 01:59:29|2017-01-

In [74]:
import requests
import json
from pyspark.sql.functions import to_timestamp, unix_timestamp, from_unixtime

# define your OpenWeatherMap API key
api_key = "7efb75d5c7c543ecbecd97c26e4fa62e"

# define the endpoint URL for the API call
endpoint_url = "https://api.openweathermap.org/data/2.5/weather"

df = df.withColumn('time_local_str', from_unixtime(df['time_local'], 'yyyy-MM-dd HH:mm:ss'))
df = df.withColumn("timestamp_local", from_unixtime(df.time_local)) \
       .withColumn("date", date_format("timestamp_local", "yyyy-MM-dd"))

# define the columns you want to join on
join_columns = ["time_local", "latitude", "longitude"]



df.show(5)


+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|         ID_Original|   latitude|   longitude|emdCardNumber|            time_utc|time_local|response_time_sec|day_of_week|weekend_or_not|            geometry|Incident_ID|       Dist_to_Seg|      XDSegID|     time_local_str|    timestamp_local|      date|
+--------------------+-----------+------------+-------------+--------------------+----------+-----------------+-----------+--------------+--------------------+-----------+------------------+-------------+-------------------+-------------------+----------+
|ObjectId(59d3a819...|36.03722849|-86.78324314|         29B5|2017-01-01 07:59:...|1483235969|            268.0|          6|             1|POINT (-86.783243...|         10|13.550370382347305|1.524393684E9|2017-01-01 01:59:29|2017-01-

In [75]:
import requests
import json
from pyspark.sql.functions import from_unixtime

# function to query the OpenWeatherMap API for weather conditions at the specified time and location
def get_weather_data(row):
    params = {
        "lat": row["latitude"],
        "lon": row["longitude"],
        "dt": row["time_local"],
        "appid": api_key,
    }
    response = requests.get(endpoint_url, params=params)
    weather_data = json.loads(response.text)
    print(response.text)
    if response is None:
      return None
    else:
      return (row["time_local"], row["latitude"], row["longitude"], weather_data.get("weather")[0].get("main"))

# create a new DataFrame with weather data joined to the original DataFrame
join_columns = ["time_local", "latitude", "longitude"]


import time

batch_size = 100
batch_count = df.count() // batch_size + 1

for i in range(batch_count):
    start_index = i * batch_size
    end_index = (i + 1) * batch_size

    # create batch dataframe
    batch_df = df.limit(batch_size).toPandas()[start_index:end_index]
    
    # process batch and join weather data
    batch_weather_df = batch_df.apply(get_weather_data, axis=1)
    batch_weather_df.columns = ["time_local", "latitude", "longitude", "weather"]
    joined_df = batch_df.join(batch_weather_df.set_index(["time_local", "latitude", "longitude"]), on=["time_local", "latitude", "longitude"])
    
    # do further processing on the joined_df
    print("next one")
    time.sleep(60) # wait for a minute between each batch to avoid exceeding the rate limit


{"cod":429, "message": "Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price"}


/content/spark-3.2.4-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


TypeError: ignored

Py4JJavaError: ignored